In [ ]:
import pandas as pd
import json
from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np
import cv2

In [ ]:
categories = []
with open("bandera-v4.0.0-categories") as f:
    categories = f.read().split("\n")

with open("test2.mp4.json") as f:
    data = json.load(f)

In [ ]:
def compute_frames_diff(vid_path, vid_name):
    print("Computing", vid_path, vid_name)
    stream = cv2.VideoCapture(vid_path + "/" + vid_name)
    success, previous_frame = stream.read()
    print(success)
    frame_count = 0

    output = open(vid_name + ".csv", "w")

    while success:
        success, frame = stream.read()
        try:
            diff = cv2.matchTemplate(frame, previous_frame, cv2.TM_CCORR_NORMED)
        except cv2.error:
            print('cv2 error at frame', frame_count)

        minVal, maxVal, minLoc, maxLoc = cv2.minMaxLoc(diff)
        output.write(str(frame_count)+";"+str(maxVal)+"\r\n")

        # cv2.imwrite("frame%d.jpg" % count, frame)     # save frame as JPEG file
        previous_frame = frame
        frame_count += 1

    output.close()

In [ ]:
print(compute_frames_diff(".", "test2.mp4"))

In [ ]:
diffs = []
with open("test2.mp4.csv") as f:
    file = f.read()
for line in file.split("\n"):
    try:
        diffs.append(float(line.split(";")[1]))
    except IndexError:
        print("ende?")
        continue
len(diffs)

In [ ]:
area = data['width']*data['height']
frames = data['frames']
per_label = defaultdict(list)

for ann in data['annotations']:
    if ann['score'] < 0.6:
        continue
    per_label[ann['category_id']].append((ann['image_id'], ann['area']/area))

#plot
fig, ax = plt.subplots(figsize=(32, 18))
    
# make sure colors don't repeat
ax.set_prop_cycle('color',plt.cm.tab20(np.linspace(0, 1, len(set(per_label.keys())))))

x_vals = np.arange(frames)

# plot frame diffs
ax.plot(range(len(diffs)), diffs)

for label, frame in per_label.items():
    x, y = zip(*frame)
    ax.plot(x, y, 'o', label=categories[label],)

ax.set_xlabel(f"Frame Number")
ax.set_ylabel(f"Percentage within frame")
ax.set_title(f"Symbols per Frame")
ax.legend()
    
plt.ylim(0, 1)

plt.show()
plt.close() 